In [54]:
team_id = '40'
split = "test_1"

In [55]:
from src.dataloader import base_loader

df, df_test = base_loader(split)

In [56]:
from src.embedding import base_embedder
from sklearn import preprocessing

X = base_embedder(df.words_str.values)
y_reg = df.score_compound.values
y_clf = preprocessing.LabelEncoder().fit_transform(df.sentiment.values)

X_test = base_embedder(df_test.words_str.values)

### Regression

In [57]:
import pandas as pd
import smogn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X_train, X_val, y_train, y_val = train_test_split(X, y_reg, test_size=0.2, random_state=42, stratify=y_clf)
# temp = pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name="y_train")], axis=1)
#
# temp_smogn = smogn.smoter(data = temp, y = "y_train")
#
# y_resampled = temp_smogn["y_train"]
# X_resampled = temp_smogn.drop["y_train"]

X_resampled = X_train
y_resampled = y_train

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [58]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor().fit(X_resampled, y_resampled)

In [59]:
from src.evaluation import eval_reg

eval_reg(reg, datasets)


Split: train
	RMSE: 0.13
	MAE: 0.10

Split: valididation
	RMSE: 0.34
	MAE: 0.26


In [60]:
from src.utils import save_sub

save_sub(reg, X_test, split, 'reg')

### Classification

In [61]:
X_train, X_val, y_train, y_val = train_test_split(X, y_clf, test_size=0.2, random_state=42, stratify=y_clf)

X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

datasets = [
    [X_resampled, y_resampled],
    [X_val, y_val]
]

In [62]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(X_resampled, y_resampled)

In [63]:
from src.evaluation import eval_clf

eval_clf(clf, datasets)


Split: train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4232
           1       1.00      1.00      1.00      4232
           2       1.00      1.00      1.00      4232

    accuracy                           1.00     12696
   macro avg       1.00      1.00      1.00     12696
weighted avg       1.00      1.00      1.00     12696


Split: validation
              precision    recall  f1-score   support

           0       0.41      0.41      0.41        96
           1       0.79      0.86      0.82      1058
           2       0.71      0.59      0.64       446

    accuracy                           0.75      1600
   macro avg       0.64      0.62      0.63      1600
weighted avg       0.75      0.75      0.75      1600



In [64]:
from src.utils import save_sub

save_sub(reg, X_test, split, 'clf')